In [50]:
import numpy as np
import cvxpy as cp

np.set_printoptions(precision=4, suppress=True)
# np.set_printoptions(formatter={"float_kind": lambda x: f"{x:.4f}"})
rng = np.random.default_rng(42)

In [51]:
n = 5  # dimension of parameters to be estimated
p = 20  # number of available types of measurements
m = 30  # number of measurements to be taken
V = rng.normal(size=(n, p))  # measurement vectors

In [52]:
np.outer(V[:, 1], V[:, 1])

array([[ 1.0816,  0.7082, -0.5649,  0.3483,  0.6884],
       [ 0.7082,  0.4637, -0.3698,  0.228 ,  0.4507],
       [-0.5649, -0.3698,  0.295 , -0.1819, -0.3595],
       [ 0.3483,  0.228 , -0.1819,  0.1121,  0.2217],
       [ 0.6884,  0.4507, -0.3595,  0.2217,  0.4381]])

In [53]:
lam = cp.Variable(p)
mats = [lam[i] * cp.outer(V[:, i], V[:, i]) for i in range(p)]
loss = cp.tr_inv(cp.sum(mats)) / m
constraints = [cp.sum(lam) == 1, lam >= 0]
prob = cp.Problem(cp.Minimize(loss), constraints)
prob.is_dcp()

True

In [54]:
prob.solve()
prob.status

'optimal'

In [55]:
lower_bound = prob.value
relaxed_optimal = lam.value
lower_bound, relaxed_optimal

(np.float64(0.24735483048207565),
 array([ 0.086 , -0.    , -0.    ,  0.0079,  0.0541, -0.    ,  0.    ,
        -0.    , -0.    , -0.    ,  0.0914,  0.086 ,  0.1863,  0.1645,
         0.1672,  0.037 , -0.    ,  0.1194,  0.    ,  0.    ]))

In [56]:
relaxed_optimal * m

array([ 2.5807, -0.0001, -0.0001,  0.2375,  1.6225, -0.    ,  0.0002,
       -0.    , -0.0001, -0.0001,  2.7434,  2.5796,  5.5902,  4.9359,
        5.0168,  1.1101, -0.0001,  3.5831,  0.0002,  0.0003])

In [57]:
np.round(relaxed_optimal * m) @ np.ones(p)

np.float64(32.0)

In [58]:
suboptimal_pt = np.around(relaxed_optimal * m)
suboptimal_pt[11] -= 1
suboptimal_pt[17] -= 1

In [59]:
suboptimal_pt

array([ 3., -0., -0.,  0.,  2., -0.,  0., -0., -0., -0.,  3.,  2.,  6.,
        5.,  5.,  1., -0.,  3.,  0.,  0.])

In [60]:
mats = [suboptimal_pt[i] * cp.outer(V[:, i], V[:, i]) for i in range(p)]
upper_bound = cp.tr_inv(cp.sum(mats)).value

In [61]:
gap = upper_bound - lower_bound
upper_bound, lower_bound, gap

(np.float64(0.24870193218252737),
 np.float64(0.24735483048207565),
 np.float64(0.0013471017004517138))